Nama : Febrian Chrissma Bagaskara <br>
NIM  : F55120059

#Koneksi Google Drive

Mengkoneksikan Ke dalam Google Drive > gdrive

In [ ]:
from google.colab import drive
# connect to Gdrive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


memilih folder Vgg16-Orange dari google drive

In [ ]:
%cd gdrive/MyDrive/Vgg16-Orange

/content/gdrive/MyDrive/Vgg16-Orange


#Prosessing Images

Memasukan library yang akan digunakan

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
     


Menginisialisasi masing-masing folder yang akan digunakan

https://drive.google.com/drive/folders/1JcAz3cY4AIs23XwTS1u5Aec_YlhI47Nb?usp=share_link

In [ ]:
data_train ='Data-Training-Jeruk/'
data_test ='Data-Testing-Jeruk/'
data_predict ='Data-Predict-Jeruk/'

Preprocessing Images untuk gambar yang akan digunakan

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Initialize Generator
gen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True, 
    rotation_range=20,
    validation_split=0.2
)

train_ds=gen.flow_from_directory(
    data_train,
    target_size= (224, 224), 
    shuffle=True,
    batch_size=32,
    class_mode='categorical',
    subset = "training"
)

validation_ds=gen.flow_from_directory(
    data_test,
    class_mode='categorical',
    shuffle=True,
    batch_size=32,
    target_size=(244,244),
    subset="validation"
)

Found 250 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


#Model CNN

Membuat model dari CNN Model

In [ ]:
# Create CNN Model
model=Sequential()

# Menambahkan blok-blok konvolusi
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Konversi matriks fitur menjadi vektor
model.add(Flatten())

# Menambahkan layer-layer fully connected
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Mengevaluasi Model yang sudah dibentuk

In [ ]:
model.fit(train_ds,epochs=10,validation_data=validation_ds, batch_size= 32, shuffle= True)

Epoch 1/10
7/8 [=========================>....] - ETA: 1:08 - loss: 64.0738 - accuracy: 0.4771

#Save Model CNN

Menyimpan Model yang sudah dibuat

#Testing Model to Predict

In [ ]:
import os
class_name=sorted(os.listdir(data_predict))
class_name

In [ ]:
IMAGE_SIZE = 224

In [ ]:
predict_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    data_predict,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    class_mode="categorical",
)

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img, 0)
    predictions = model.predict(img_array)
    predicted_class = class_name[np.argmax(predictions)]
    confidence = round(100 * (np.max(predictions)), 1)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))

for images, labels in predict_gen:
    for i in range(6):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        predicted_class, confidence = predict(model, images[i])
        actual_class = class_name[int(labels[i][1])] 
        plt.title(f"Aktual: {actual_class},\n Prediksi: {predicted_class},\n Akurasi: {confidence}%")
        plt.axis("off")
    break